In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install neattext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import joblib
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import os

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Final Year Project Work/Dataset/emotion_dataset_raw.csv")

In [ ]:
import neattext.functions as nfx
df['Clean_Text'] = df['Text'].apply(nfx.remove_userhandles)
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

In [ ]:
Xfeatures = df['Clean_Text']
ylabels = df['Emotion']
x_train, x_test, y_train, y_test = train_test_split(Xfeatures, ylabels, test_size=0.3, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
max_seq_length = 128

def tokenize_texts(texts, max_length):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            padding='max_length',
                            truncation=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks

x_train_ids, x_train_masks = tokenize_texts(x_train, max_seq_length)
x_test_ids, x_test_masks = tokenize_texts(x_test, max_seq_length)

In [ ]:
for i, text in enumerate(x_train[:5]):  # Print first 5 tokenized samples
    print(f"Original Text: {text}")
    print(f"Tokenized Input IDs: {x_train_ids[i]}")
    print(f"Attention Mask: {x_train_masks[i]}")
    print("=" * 80)

Original Text: Dressing tomorrow basketball :)
Tokenized Input IDs: tensor([  101, 11225,  4826,  3455,  1024,  1007,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,    

In [ ]:
class BERTbiLSTMModel(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes):
        super(BERTbiLSTMModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.bilstm = nn.LSTM(input_size=bert_model.config.hidden_size,
                              hidden_size=hidden_size,
                              num_layers=1,
                              batch_first=True,
                              bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        bilstm_output, _ = self.bilstm(pooled_output.unsqueeze(1))  # Add an additional dimension
        logits = self.fc(bilstm_output[:, -1, :])  # Take the last hidden state
        return logits


bert_model = BertModel.from_pretrained('bert-base-uncased')
num_classes = len(df['Emotion'].unique())
hidden_size = 128
model = BERTbiLSTMModel(bert_model, hidden_size, num_classes)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Convert emotion labels to numerical format
emotion_to_label = {emotion: label for label, emotion in enumerate(df['Emotion'].unique())}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

y_train_tensor = torch.tensor([emotion_to_label[emotion] for emotion in y_train.values])
y_test_tensor = torch.tensor([emotion_to_label[emotion] for emotion in y_test.values])

batch_size = 32
train_dataset = TensorDataset(x_train_ids, x_train_masks, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(x_test_ids, x_test_masks, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for input_ids, attention_mask, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Average training loss: {avg_train_loss:.4f}')

torch.save(model.state_dict(), '/content/drive/MyDrive/Final Year Project Work/Model Weights/emotion_classifier_bert_bilstm.pt')

Epoch 1/15: 100%|██████████| 762/762 [07:35<00:00,  1.67it/s]


Average training loss: 1.3389


Epoch 2/15: 100%|██████████| 762/762 [07:49<00:00,  1.62it/s]


Average training loss: 0.8848


Epoch 3/15: 100%|██████████| 762/762 [07:50<00:00,  1.62it/s]


...

Average training loss: 0.0714


Epoch 14/15: 100%|██████████| 762/762 [07:54<00:00,  1.60it/s]


Average training loss: 0.0733


Epoch 15/15: 100%|██████████| 762/762 [07:54<00:00,  1.60it/s]


Average training loss: 0.0537


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn


bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class BERTbiLSTMModel(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes):
        super(BERTbiLSTMModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.bilstm = nn.LSTM(input_size=bert_model.config.hidden_size,
                              hidden_size=hidden_size,
                              num_layers=1,
                              batch_first=True,
                              bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        bilstm_output, _ = self.bilstm(pooled_output.unsqueeze(1))  # Add an additional dimension
        logits = self.fc(bilstm_output[:, -1, :])  # Take the last hidden state
        return logits

# Recreate the model with the correct parametersnum_classes = 8
hidden_size = 128
model = BERTbiLSTMModel(bert_model, hidden_size, num_classes)

# Load the state dictionary
model.load_state_dict(torch.load('/content/drive/MyDrive/Final Year Project Work/Model Weights/emotion_classifier_bert_bilstm.pt', map_location=torch.device('cpu')))
model.eval()

# Move the model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # Move the entire model to the selected device


# Function to predict emotion
def predict_emotion(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True, padding='max_length')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        print(probabilities)
        predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class

# Testing
sentence = "I feel so embarrassed about what I did yesterday"
predicted_emotion = predict_emotion(sentence)

# label to emotion
label_to_emotion = {label: emotion for emotion, label in emotion_to_label.items()}



predicted_emotion_label = label_to_emotion[predicted_emotion]

print("Predicted Emotion Label:", predicted_emotion_label)

tensor([[1.4788e-03, 7.9841e-04, 4.4817e-03, 1.4503e-03, 5.4556e-03, 2.2966e-03,
         9.6508e-01, 1.8957e-02]])
Predicted Emotion Label: shame


In [ ]:
emotion_classes = df['Emotion'].unique()
print(emotion_classes)

['neutral' 'joy' 'sadness' 'fear' 'surprise' 'anger' 'shame' 'disgust']


In [ ]:
!python --version

Python 3.11.11


In [ ]:
classifier("It was a totally unexpected twist!")

[{'label': 'surprise', 'score': 0.7009335160255432}]

In [ ]:
import pickle

In [ ]:
with open("/content/drive/MyDrive/Final Year Project Work/Model Weights/english_emotion_text_classifier.pkl", "wb") as f:
    pickle.dump(classifier, f)

In [ ]:
# Load the model from the pickle file
with open("/content/drive/MyDrive/Final Year Project Work/Model Weights/english_emotion_text_classifier.pkl", "rb") as f:
    loaded_classifier = pickle.load(f)

# Use the loaded classifier
result = loaded_classifier("It was a totally unexpected twist!")
print(result)

[{'label': 'surprise', 'score': 0.7009335160255432}]
